In [1]:
# based on this project: https://www.blog.datahut.co/post/scraping-amazon-best-seller-data-using-python

# Importing libraries
import time
import random
import csv  
from csv import writer
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None
import undetected_chromedriver as uc #driver that can't be detected by websites as a scraper
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Function to delay some process
def delay():
    # time.sleep(random.randint(1, 3))
    time.sleep(1)

# Scrolling down the page in order to overcome Lazy Loading
def lazy_loading(n):
    element = driver.find_element(By.TAG_NAME, 'body')
    count = 0
    while count < n:
        element.send_keys(Keys.PAGE_DOWN)
        delay()
        count += 1

In [29]:
# Function to fetch the product links of products
def fetch_product_links_and_ranks():
    content = driver.page_source
    homepage_soup = BeautifulSoup(content, 'html.parser')

    all_products = homepage_soup.find('div', attrs={"class": "p13n-desktop-grid"})
    for product_section in all_products.find_all('div', {'id': 'gridItemRoot'}):
        for product_link in product_section.find_all('a',{'tabindex':'-1'}):

            if product_link['href'].startswith('https:'):
                url_to_be_saved = product_link['href']
            else:
                url_to_be_saved = 'https://www.amazon.com.mx' + product_link['href']
            # print("obtained url: ", url_to_be_saved)
            
            df = pd.read_csv('links_amazon.csv')
            if url_to_be_saved not in df.product_url.values:

                # insert row of information to .csv file
                data = [df.number.iloc[-1] + 1, url_to_be_saved, "no"]
                # Open our existing CSV file in append mode and Create a file object for this file
                with open('links_amazon.csv', 'a', encoding="utf-8", newline='') as f_object:
                
                    # Pass this file object to csv.writer() and get a writer object
                    writer_object = writer(f_object)
                
                    # Pass the list as an argument into the writerow() and Close the file object
                    writer_object.writerow(data)
                    f_object.close()

options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\j50022283\AppData\Local\Google\Chrome\User Data\Default")

driver = uc.Chrome(options=options)
# driver = webdriver.Chrome()

# Fetching the product links of all items
for page in range(1,3): # to iterate over the 2 pages in which the products are divided into
    start_url = f'https://www.amazon.com.mx/gp/bestsellers/books/ref=zg_bs_pg_{page}_books?ie=UTF8&pg={page}'

    driver.get(start_url)
    print("Scrapping main", start_url)
    lazy_loading(16)                # to overcome lazy loading
    fetch_product_links_and_ranks() # to fetch the links to products 


Scrapping main https://www.amazon.com.mx/gp/bestsellers/books/ref=zg_bs_pg_1_books?ie=UTF8&pg=1
Scrapping main https://www.amazon.com.mx/gp/bestsellers/books/ref=zg_bs_pg_2_books?ie=UTF8&pg=2


In [43]:
df = pd.read_csv ('links_amazon.csv')

# print(df)
print(df.already_scraped.value_counts())

yes    90
no     10
Name: already_scraped, dtype: int64


In [44]:
import re

options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\j50022283\AppData\Local\Google\Chrome\User Data\Default") #change this to your profile path, you can get it on google in this link: 'chrome://version/'

driver = uc.Chrome(options=options)
# driver = webdriver.Chrome()

for index in df.index:
    url = df.product_url[index]

    # if index == 60:break

    if df.already_scraped[index] == 'no':
        driver.get(url)
        page_content = driver.page_source
        soup = BeautifulSoup(page_content, 'html.parser')
        print("\n" + str(df.number[index]) + ".- The following URL is being scrapped out: ", url)
        
        # getting name
        try:
            name_of_product = soup.find('span', attrs={"id": "productTitle"}).text.strip()
        except:
            name_of_product = 'unavailable'
        print("name_of_product:",name_of_product)

        # getting stars
        try:
            rating_stars = soup.find('span', attrs={"id": "acrPopover"})["title"]
        except:
            rating_stars = 'unavailable'
        print("rating_stars:",rating_stars)

        # getting total number of reviews
        try:
            Customer_Reviews = soup.find('span', attrs={"id": "acrCustomerReviewText"}).text.strip()
        except:
            Customer_Reviews = 'unavailable'
        print("Customer_Reviews:",Customer_Reviews)

        # getting description of books (several paragraphs)
        try:
            description_of_product = soup.find('div', attrs={"class": "a-expander-content a-expander-partial-collapse-content"})
            description_of_product = description_of_product.find_all('span')
            description = []
            descriptionString = ""
            for span in description_of_product:
                description.append(span.text.strip())
            descriptionString = ' \n'.join(description)
            print("descriptionString:",description)
        except:
            descriptionString = 'unavailable'
            print("descriptionString:",descriptionString)

        # getting specific details of book
        try:
            details_of_product = soup.find('div', attrs={"id": "detailBullets_feature_div"})
            details_of_product = details_of_product.find_all('span', attrs={"class": "a-list-item"})
            detailsA = []
            detailsB = []
            detailsString = ""
            for span in details_of_product:
                a = span.find('span', attrs={"class": "a-text-bold"})
                b = span.find('span', attrs={"class": ""})
                a = re.sub('\u200f\n', '', str(a))
                a = re.sub('\u200e\n', '', str(a))
                a = re.sub('\n', '', str(a))
                a = re.sub(':', '', str(a))
                a = BeautifulSoup(str(a), 'html.parser').text.strip()
                b = BeautifulSoup(str(b), 'html.parser').text.strip()

                detailsA.append(a)
                if b != None and b != 'None': 
                    detailsB.append(b)

            detailsC = []
            for i in range(len(detailsB)):
                detailsC.append(str(detailsA[i]) + ' : ' + str(detailsB[i]))
            detailsString = ' \n'.join(detailsC)
            
            # print("detailsStringA:",detailsA)
            # print("detailsStringB:",detailsB)
            # print("detailsStringC:",detailsC)
            print("detailsString:",detailsString)
        except:
            detailsString = 'unavailable'
            print("detailsString:",detailsString)
            
        # getting author (more than 1 author migth appear)
        try:
            # authors_detail = soup.find('div', attrs={"id": "moreAboutTheAuthorCard_feature_div"})
            # authors_detail = authors_detail.find_all('h2')

            # if not authors_detail: #sometimes the previous div is empty so we have to try with this one
            authors_detail = soup.find('div', attrs={"id": "bylineInfo_feature_div"})
            authors_detail = authors_detail.find_all('span', attrs={"class": "author notFaded"})

            authors = []
            authorsString = ""
            for author in authors_detail:
                authors.append(author.text.strip())

            if authors[0] == 'Acerca del autor' or authors[0] == 'Acerca de los autores':
                authorsString = ' '.join(authors[1:])
            else:
                for i in range(len(authors)):
                    authors[i] = re.sub('\n', '', authors[i])
                authorsString = ' '.join(authors)

            print("authorsString:",authorsString)

        except:
            authorsString =  'unavailable'
            print("authorsString:", authorsString)

        # getting url of comments to scrape them out later on
        try:
            all_reviews_link = 'https://www.amazon.com.mx' + soup.find('a', attrs={"data-hook": "see-all-reviews-link-foot"})["href"]
        except:
            all_reviews_link = 'unavailable'
        print("all_reviews_link:",all_reviews_link)


        # insert row of information to .csv file
        data = [df.number[index], name_of_product, rating_stars, Customer_Reviews, descriptionString, detailsString, authorsString, all_reviews_link, "no"]
        
        # Open our existing CSV file in append mode and Create a file object for this file
        with open('books_amazon.csv', 'a', encoding="utf-8", newline='') as csvfile:
        
            # Pass this file object to csv.writer() and get a writer object
            writer_object = writer(csvfile)
        
            # Pass the list as an argument into the writerow() and Close the file object
            writer_object.writerow(data)
            csvfile.close()

        if data:
            df.already_scraped[index] = 'yes'
            df.to_csv('links_amazon.csv', index=False)



0.- The following URL is being scrapped out:  https://www.amazon.com.mx/Ragnarok-Camino-Hombre-Hombres-Peligrosos/dp/B0BCSGPCT6/ref=zg_bs_g_books_sccl_91/137-7391295-0351748?psc=1
name_of_product: Ragnarok: El Camino de un Hombre
rating_stars: 4.6 de 5 estrellas
Customer_Reviews: 190 calificaciones
descriptionString: ['¡HEY TÚ! DEBES LEER ESTO SI ERES HOMBRE.', 'Vivimos en una sociedad donde es más que nunca necesario forjar masculinidad. La sociedad de hoy te quiere débil, te quiere manipulable, te quiere desesperado, pero sobre todo te quiere dormido.', 'Tú demuestra ser fuerte, masculino, campeón, calmado, observador y despierto. Este libro no va de seducción, este libro va de que abras los ojos y veas la realidad de la mujer moderna de "Alto Valor" que no tiene nada que ofrecer más que "su belleza física", claro que hay algunas mujeres que valen la pena y en este libro te daremos las claves para evitar las que no te aportan valor y que forjes tus cualidades como hombre.', 'Estoy s

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.142)
Stacktrace:
	GetHandleVerifier [0x00A437C3+48947]
	(No symbol) [0x009D8551]
	(No symbol) [0x008DC92D]
	(No symbol) [0x008C36E5]
	(No symbol) [0x00928CDB]
	(No symbol) [0x00937113]
	(No symbol) [0x00924E86]
	(No symbol) [0x009016C7]
	(No symbol) [0x0090284D]
	GetHandleVerifier [0x00C8FDF9+2458985]
	GetHandleVerifier [0x00CD744F+2751423]
	GetHandleVerifier [0x00CD1361+2726609]
	GetHandleVerifier [0x00AC0680+560624]
	(No symbol) [0x009E238C]
	(No symbol) [0x009DE268]
	(No symbol) [0x009DE392]
	(No symbol) [0x009D10B7]
	BaseThreadInitThunk [0x76B500C9+25]
	RtlGetAppContainerNamedObjectPath [0x777F7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x777F7B1E+238]


In [5]:
#just run this cell in case of wanting to erase all links_amazon.csv content 
header = ['number', 'product_url', 'already_scraped']

with open('links_amazon.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

In [29]:
#just run this cell in case of wanting to erase all books_amazon.csv content 

header = ['number', 'name', 'stars', 'number_of_reviews', 'description', 'details', 'author', 'opinions_url', 'opinions_already_scraped']

with open('books_amazon.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

df = pd.read_csv('links_amazon.csv')
df.already_scraped = 'no'
df.to_csv('links_amazon.csv', index=False)

In [35]:
#just run this cell in case of wanting to erase all opinions_amazon.csv content 
header = ['number', 'name', 'stars', 'title', 'date', 'text']

with open('opinions_amazon.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

df = pd.read_csv('books_amazon.csv')
df.opinions_already_scraped = 'no'
df.to_csv('books_amazon.csv', index=False)

In [8]:
df = pd.read_csv('books_amazon.csv')

print(df)

    number                                               name  \
0        0                Diccionario Pocket (Español-Inglés)   
1        0  Este dolor no es mío. Identifica y resuelve lo...   
2        0                                   Hábitos Atómicos   
3        0                             Las 48 leyes del poder   
4        0                      Descubre a los mejores amigos   
5        0        Beyond the Story. Crónica de 10 años de BTS   
6        0                     El niño que tocó las estrellas   
7        0  El método Silva de control mental. La vida que...   
8        0  Netter. Cuaderno de anatomía para colorear (3ª...   
9        0  Los Cuatro Acuerdos: Un libro de sabiduría tol...   
10       0                     La Biblioteca De La Medianoche   
11       0                        La revolución de la glucosa   
12       0  Baby Einstein - Mi Primera Biblioteca - 12 Lib...   
13       0                      Encuentra tu persona vitamina   
14       0  Cómo hacer qu

In [27]:
# getting opinions

options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\j50022283\AppData\Local\Google\Chrome\User Data\Default")

driver = uc.Chrome(options=options)
# driver = webdriver.Chrome()

for index in df.index:
    if index == 10:break

    url = df.opinions_url[index]

    if df.opinions_already_scraped[index] == 'no' and df.opinions_url[index] != "unavailable":
        print("\n" + str(df.number[index]) + ".- The following URL is being scrapped out: ", url)
    
        driver.get(url)
        lazy_loading(8)
        
        page_content = driver.page_source
        soup = BeautifulSoup(page_content, 'html.parser')
        
        
        # getting opinions
        try:
            opinions = soup.find('div', attrs={"class": "a-section a-spacing-none reviews-content a-size-base"})
            opinions = opinions.find_all('div', attrs={"class": "a-section review aok-relative"})

            for opinion in opinions:
                # print(opinion)
                try:
                    name = opinion.find('span', attrs={"class": "a-profile-name"}).text.strip()
                except:
                    name = "unavailable"

                try: 
                    divRating_and_title = opinion.find('div', attrs={"class": "a-row"})
                    # print("a row", divRating_and_title)
                    rating = divRating_and_title.find('span', attrs={"class": "a-icon-alt"}).text.strip()
                    title = divRating_and_title.find('span', attrs={"class": ""}).text.strip()
                except:
                    rating = "unavailable"
                    title = "unavailable"

                try: 
                    opinion_date = opinion.find('span', attrs={"class": "a-size-base a-color-secondary review-date"}).text.strip()
                except:
                    opinion_date = "unavailable"

                try:   
                     divRating_opinion = opinion.find('div', attrs={"class": "a-row a-spacing-small review-data"})
                    #  print("divRating_opinion", divRating_opinion)
                     opinion_text = divRating_opinion.find('span', attrs={"class": ""}).text.strip()
                except:
                    opinion_text = "unavailable"

                # print("\n")
                # print(name)
                # print(rating)
                # print(title)
                # print(opinion_date)
                # print(opinion_text)
                
                # df.number[index]
                data = [index, name, rating, title, opinion_date, opinion_text]
                print("\n", data)
        
                # Open our existing CSV file in append mode and Create a file object for this file
                with open('opinions_amazon.csv', 'a', encoding="utf-8", newline='') as f_object:
                
                    # Pass this file object to csv.writer() and get a writer object
                    writer_object = writer(f_object)
                
                    # Pass the list as an argument into the writerow() and Close the file object
                    writer_object.writerow(data)
                    f_object.close()

                if data:
                    df.opinions_already_scraped[index] = 'yes'
                    df.to_csv('books_amazon.csv', index=False)
                    

        except:
            opinions = 'opinions not available for this page'
            print(opinions)


0.- The following URL is being scrapped out:  https://www.amazon.com.mx/Larousse-Diccionario-Pocket-Espanol-Ingles-Ingles-Espanol/product-reviews/9706074937/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


jose rodriguez
5.0 de 5 estrellas
excelente diccionario
Revisado en México el 20 de agosto de 2023
el diccionario realmente no es tan pequeño , digamos que tiene un tamaño estandar de unas 8 o 9 pulgadas, las letras a pesdar de que son pequeñas son muy legibles y lo digo yo que uso lentes sin problema puedo leer lo que dice , no es de pasta gruesa y las hojas con algo delicadas pero realmente para un uso escolar que es por lo que lo compre esta perfecto

 [0, 'jose rodriguez', '5.0 de 5 estrellas', 'excelente diccionario', 'Revisado en México el 20 de agosto de 2023', 'el diccionario realmente no es tan pequeño , digamos que tiene un tamaño estandar de unas 8 o 9 pulgadas, las letras a pesdar de que son pequeñas son muy legibles y lo digo yo que uso lentes sin problema

In [27]:
# mercado libre scraping, just test

options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\j50022283\AppData\Local\Google\Chrome\User Data\Default")

driver = uc.Chrome(options=options)
# driver = webdriver.Chrome()


url = "https://www.mercadolibre.com.mx/la-teoria-del-todo-el-origen-y-el-destino-del-universo-de-hawking-stephen-serie-ensayo-editorial-debolsillo-tapa-blanda-en-espanol-2010/p/MLM21056700?matt_tool=52069260&matt_word=&matt_source=google&matt_campaign_id=19652628393&matt_ad_group_id=143232870702&matt_match_type=&matt_network=g&matt_device=c&matt_creative=647515868841&matt_keyword=&matt_ad_position=&matt_ad_type=pla&matt_merchant_id=735123861&matt_product_id=MLM21056700-product&matt_product_partition_id=1816346066369&matt_target_id=aud-2038635960613:pla-1816346066369&gclid=Cj0KCQjw3JanBhCPARIsAJpXTx7q0j-A5J3lxRuaf0fsXJ9OT-9Ad5ROffJw-PuvEWkyRwLVxCumYokaArI9EALw_wcB"

driver.get(url)
page_content = driver.page_source
soup = BeautifulSoup(page_content, 'html.parser')
print("\n" + "The following URL is being scraped out: ", url)

# getting name
try:
    name_of_product = soup.find('h1', attrs={"class": "ui-pdp-title"}).text.strip()
except:
    name_of_product = 'Product name not available '
print(name_of_product)

# getting features
try:
    features = soup.find('table', attrs={"class": "andes-table"})
    features = features.find_all('span', attrs={"class": "andes-table__column--value"})
except:
    features = 'features not available '
print(features)

# getting description
try:
    description = soup.find('p', attrs={"class": "ui-pdp-description__content"}).text.strip()
except:
    description = 'desc not available '
print(description)


lazy_loading(6)

button_show_opinions = driver.find_element(by=By.CLASS_NAME, value="show-more-click")
button_show_opinions.click()

time.sleep(5)
element_inside_popup = driver.find_element(by=By.CLASS_NAME, value="infinite-scroll-component ")
element_inside_popup.send_keys(Keys.PAGE_DOWN)
element_inside_popup.send_keys(Keys.PAGE_DOWN)
element_inside_popup.send_keys(Keys.END)

# lazy_loading(5)

page_content = driver.page_source
soup = BeautifulSoup(page_content, 'html.parser')
print("scraping opinions out: ")

# getting opinions
try:
    opinions = soup.find_all('p', attrs={"class": "ui-review-capability-comments__comment__content"})
except:
    opinions = 'Product name not available '
print(opinions)



The following URL is being scraped out:  https://www.mercadolibre.com.mx/la-teoria-del-todo-el-origen-y-el-destino-del-universo-de-hawking-stephen-serie-ensayo-editorial-debolsillo-tapa-blanda-en-espanol-2010/p/MLM21056700?matt_tool=52069260&matt_word=&matt_source=google&matt_campaign_id=19652628393&matt_ad_group_id=143232870702&matt_match_type=&matt_network=g&matt_device=c&matt_creative=647515868841&matt_keyword=&matt_ad_position=&matt_ad_type=pla&matt_merchant_id=735123861&matt_product_id=MLM21056700-product&matt_product_partition_id=1816346066369&matt_target_id=aud-2038635960613:pla-1816346066369&gclid=Cj0KCQjw3JanBhCPARIsAJpXTx7q0j-A5J3lxRuaf0fsXJ9OT-9Ad5ROffJw-PuvEWkyRwLVxCumYokaArI9EALw_wcB
La teoría del todo: El origen y el destino del universo, de Hawking, Stephen. Serie Ensayo Editorial Debolsillo, tapa blanda en español, 2010
[<span class="andes-table__column--value" style="line-clamp:none;-webkit-line-clamp:none">La teoría del todo</span>, <span class="andes-table__column--

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=116.0.5845.141)
Stacktrace:
	GetHandleVerifier [0x00F937C3+48947]
	(No symbol) [0x00F28551]
	(No symbol) [0x00E2C7F0]
	(No symbol) [0x00E56458]
	(No symbol) [0x00E55AB5]
	(No symbol) [0x00E7508C]
	(No symbol) [0x00E52EB6]
	(No symbol) [0x00E753E4]
	(No symbol) [0x00E875DA]
	(No symbol) [0x00E74E86]
	(No symbol) [0x00E516C7]
	(No symbol) [0x00E5284D]
	GetHandleVerifier [0x011DFDF9+2458985]
	GetHandleVerifier [0x0122744F+2751423]
	GetHandleVerifier [0x01221361+2726609]
	GetHandleVerifier [0x01010680+560624]
	(No symbol) [0x00F3238C]
	(No symbol) [0x00F2E268]
	(No symbol) [0x00F2E392]
	(No symbol) [0x00F210B7]
	BaseThreadInitThunk [0x76A900C9+25]
	RtlGetAppContainerNamedObjectPath [0x77A57B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77A57B1E+238]
